# Data Cleaning

Our first strategy to figure out how to clean the data was to simply look at it in an excel spreadsheet. 
Here are some things we noticed:
* Lots of the categories are non-numerical (breed, climate zone, management system, date, etc.)
* lots of numerical values start with a "'"
* Some negative values don't make sense (rumination time)
* Some blank values
* Feed quantity is present twice

In [2]:
# imports 

import pandas as pd 
import sklearn 

In [13]:
# read in the data
data = pd.read_csv("cattle_data_train.csv")
test = pd.read_csv("cattle_data_test.csv")
labels = data.iloc()[:, -1]
data.head()

,Cattle_ID,Breed,Climate_Zone,Management_System,Age_Months,Weight_kg,Parity,Lactation_Stage,Days_in_Milk,Feed_Type,...,BVD_Vaccine,Rabies_Vaccine,Previous_Week_Avg_Yield,Body_Condition_Score,Milking_Interval_hrs,Date,Farm_ID,Feed_Quantity_lb,Mastitis,Milk_Yield_L
0,CATTLE_133713,Holstein,Tropical,Intensive,114,544.8,4,Mid,62,Concentrates,...,0,1,6.31,3.0,12,2024-01-15,FARM_0301,36.8235,1,12.192634
1,CATTLE_027003,Holstein,Arid,Mixed,136,298.9,4,Mid,213,Crop_Residues,...,0,0,17.16,4.0,12,2023-10-31,FARM_0219,NaN,0,14.717031
2,CATTLE_122459,Holstein,Tropical,Semi_Intensive,64,336.6,4,Late,16,Hay,...,1,0,4.07,3.5,12,2024-05-20,FARM_0802,16.0965,0,14.006142
3,CATTLE_213419,Jersey,Mediterranean,Intensive,58,370.5,1,Early,339,Crop_Residues,...,0,0,10.23,3.0,24,2024-07-22,FARM_0034,40.7925,0,24.324325
4,CATTLE_106260,Guernsey,Subtropical,Intensive,84,641.5,6,Early,125,Mixed_Feed,...,1,1,20.68,3.0,12,2023-01-03,FARM_0695,33.7365,1,12.023074


In [14]:
# checking which columns have missing values

data.isna().sum()

Cattle_ID                      0
Breed                          0
Climate_Zone                   0
Management_System              0
Age_Months                     0
Weight_kg                      0
Parity                         0
Lactation_Stage                0
Days_in_Milk                   0
Feed_Type                      0
Feed_Quantity_kg           10481
Feeding_Frequency              0
Water_Intake_L                 0
Walking_Distance_km            0
Grazing_Duration_hrs           0
Rumination_Time_hrs            0
Resting_Hours                  0
Ambient_Temperature_C          0
Humidity_percent               0
Housing_Score               6279
FMD_Vaccine                    0
Brucellosis_Vaccine            0
HS_Vaccine                     0
BQ_Vaccine                     0
Anthrax_Vaccine                0
IBR_Vaccine                    0
BVD_Vaccine                    0
Rabies_Vaccine                 0
Previous_Week_Avg_Yield        0
Body_Condition_Score           0
Milking_In

In [19]:
# Number of duplicates in each column

# data.apply(lambda s: s.duplicated().sum())
data.apply(lambda s: s.value_counts().gt(1).sum())


Cattle_ID                    34
Breed                         7
Climate_Zone                  6
Management_System             5
Age_Months                  120
Weight_kg                  5001
Parity                        6
Lactation_Stage               3
Days_in_Milk                364
Feed_Type                     8
Feed_Quantity_kg              0
Feeding_Frequency             5
Water_Intake_L                0
Walking_Distance_km        1028
Grazing_Duration_hrs        131
Rumination_Time_hrs        2937
Resting_Hours               131
Ambient_Temperature_C         0
Humidity_percent            901
Housing_Score                 0
FMD_Vaccine                   2
Brucellosis_Vaccine           2
HS_Vaccine                    2
BQ_Vaccine                    2
Anthrax_Vaccine               2
IBR_Vaccine                   2
BVD_Vaccine                   2
Rabies_Vaccine                2
Previous_Week_Avg_Yield    2937
Body_Condition_Score          7
Milking_Interval_hrs          4
Date    

In [29]:
# We are considering how to impute the missing values for housing score and feed amount.
# One thought was that maybe all the cows from one farm are missing a housing score/feed amount. 
# We found here that that is not the case, as the max missing from any given 
# farm is 22 for either one. We now explore using the mean housing score or feed amount from a given 
# farm to impute missing values. Intuitively, both of these make sense, however, we 
# find that while, within farms, housing score varies very little (VAR: 0.04100923726054576)
# while feed amount varies a lot: (VAR: 15.74869934892377). 
# Unfortunately, we found that the variances across the entire data set (15.754920313482826, 0.04101214069288791)
# are very similar, as were the means, generally, so it would not help very much
# to impute with the mean of the farms. 



# score values is to take the average from the farm, which we will do in the next cell. 

max_missing = -1
total_housing_var = 0
total_feed_var = 0

for name, group in data.groupby("Farm_ID"):
    # Missing counts
    missing_housing = group["Housing_Score"].isna().sum()
    missing_feed = group["Feed_Quantity_kg"].isna().sum()
    
    total = len(group)

    # Track max missing across the two columns
    max_missing = max(max_missing, missing_housing, missing_feed)

    # Only print groups with any missing values
    if missing_housing > 0 or missing_feed > 0:

        # Variances (skip NaN automatically)
        housing_var = group["Housing_Score"].var()
        feed_var = group["Feed_Quantity_kg"].var()
        housing_mean = group["Housing_Score"].mean()
        feed_mean = group["Feed_Quantity_kg"].mean()

        total_housing_var += housing_var
        total_feed_var += feed_var

        print(f"Group: {name}")
        print(f"  Missing Housing_Score:     {missing_housing}")
        print(f"  Missing Feed_Quantity_kg:  {missing_feed}")
        print(f"  Total in group:            {total}")

        print(f"  Housing_Score variance:    {housing_var}")
        print(f"  Feed_Quantity_kg variance: {feed_var}\n")
        print(f"  Housing_Score mean:    {housing_mean}")
        print(f"  Feed_Quantity_kg mean: {feed_mean}\n")


average_feed_var = total_feed_var / 1000
average_housing_var = total_housing_var / 1000 

whole_data_feed_var = data["Feed_Quantity_kg"].var()
whole_data_housing_var = data["Housing_Score"].var()

print("whole data feed var: " + str(whole_data_feed_var))
print("whole data housing var: " + str(whole_data_housing_var))
print("average feed var: " + str(average_feed_var) + "\naverage housing var: " + str(average_housing_var))
print("max missing from any given farm: " + str(max_missing))

Group: FARM_0001
  Missing Housing_Score:     7
  Missing Feed_Quantity_kg:  13
  Total in group:            199
  Housing_Score variance:    0.04217811540431952
  Feed_Quantity_kg variance: 15.355051079652638

  Housing_Score mean:    0.6506222698428817
  Feed_Quantity_kg mean: 12.346106829669043

Group: FARM_0002
  Missing Housing_Score:     4
  Missing Feed_Quantity_kg:  10
  Total in group:            200
  Housing_Score variance:    0.04123126975618914
  Feed_Quantity_kg variance: 17.112276135559075

  Housing_Score mean:    0.6765135181913958
  Feed_Quantity_kg mean: 12.06853684385468

Group: FARM_0003
  Missing Housing_Score:     7
  Missing Feed_Quantity_kg:  11
  Total in group:            204
  Housing_Score variance:    0.04041616495527874
  Feed_Quantity_kg variance: 15.307978398294088

  Housing_Score mean:    0.6281224369763271
  Feed_Quantity_kg mean: 12.362122895737603

Group: FARM_0004
  Missing Housing_Score:     5
  Missing Feed_Quantity_kg:  15
  Total in group:    

In [33]:

# Here we see that the correlation between housing quality and feed amount
# and all other data is quite low, meaning this would be a poor way to impute the data. 


# Select only numeric columns (automatically ignores text columns)
numeric_cols = data.select_dtypes(include=['number'])

# Compute correlations with Feed_Quantity_KG
corr_with_feed = numeric_cols.corr()['Feed_Quantity_kg'].dropna()
corr_with_housing = numeric_cols.corr()['Housing_Score'].dropna()
print(corr_with_feed)
print("\n\n")
print(corr_with_housing)

Age_Months                 0.000363
Weight_kg                 -0.002157
Parity                     0.004293
Days_in_Milk               0.001656
Feed_Quantity_kg           1.000000
Feeding_Frequency          0.001352
Water_Intake_L             0.002390
Walking_Distance_km        0.000100
Grazing_Duration_hrs      -0.000462
Rumination_Time_hrs        0.040673
Resting_Hours             -0.001615
Ambient_Temperature_C      0.000249
Humidity_percent           0.001180
Housing_Score              0.000339
FMD_Vaccine               -0.001442
Brucellosis_Vaccine       -0.001925
HS_Vaccine                -0.000649
BQ_Vaccine                 0.000783
Anthrax_Vaccine            0.001621
IBR_Vaccine                0.000923
BVD_Vaccine               -0.001813
Rabies_Vaccine             0.000916
Previous_Week_Avg_Yield    0.040673
Body_Condition_Score      -0.000161
Milking_Interval_hrs      -0.000398
Feed_Quantity_lb           0.998757
Mastitis                   0.003009
Milk_Yield_L               0